## Домашнее задание №4

<ol>
    <li>Попробовать улучшить точность распознования образов cifar 10 сверточной нейронной сетью, рассмотренной на уроке. Приложить анализ с описанием того, что улучшает работу нейронной сети и что ухудшает.
    </li>
    <li>Описать также в анализе какие необоходимо внести изменения  в получившуюся у вас нейронную сеть если бы ей нужно было работать не с cifar10, а с MNIST, CIFAR100 и IMAGENET.
    </li>
</ol>

In [1]:
from __future__ import print_function
import keras 
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

# установка параметров нейросети
batch_size = 64
num_classes = 10
epochs = 5
data_augmentation = True
num_predictions = 50
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# разделение тренировочной и тестовой выборки
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'тренировочные примеры')
print(x_test.shape[0], 'тестовые примеры')

# преобразование матрицы чисел 0-9 в бинарную матрицу чисел 0-1
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# полносвязные слои нейронной сети
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# инициализация RMSprop optimizer
opt = keras.optimizers.RMSprop(learning_rate=0.001, decay=1e-6)

# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Не используется data augmentation')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Использование data augmentation в реальном времени')
    # Препроцессинг и data augmentation в реальном времени:
    datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False, 
        zca_epsilon=1e-06, 
        rotation_range=0, 
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0., 
        zoom_range=0., 
        channel_shift_range=0.,
        fill_mode='nearest',
        cval=0.,
        horizontal_flip=True,
        vertical_flip=False,
        rescale=None,
        preprocessing_function=None,
        data_format=None,
        validation_split=0.0)

    # запуск data augmentation через fit
    #datagen.fit(x_train)

    # запуск data augmentation через fit_generator
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                                     epochs=epochs,
                                     validation_data=(x_test, y_test),
                                     workers=4)

# сохранение модели и весов
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('сохранить обученную модель как %s ' % model_path)

# проверка работы обученной модели
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Using TensorFlow backend.


x_train shape: (50000, 32, 32, 3)
50000 тренировочные примеры
10000 тестовые примеры
Использование data augmentation в реальном времени
Epoch 1/5
782/782 [==============================] - 549s 702ms/step - loss: 1.7087 - accuracy: 0.3791 - val_loss: 1.2997 - val_accuracy: 0.5336
Epoch 2/5
782/782 [==============================] - 549s 703ms/step - loss: 1.3223 - accuracy: 0.5302 - val_loss: 1.0456 - val_accuracy: 0.6240
Epoch 3/5
782/782 [==============================] - 563s 720ms/step - loss: 1.1892 - accuracy: 0.5787 - val_loss: 1.0494 - val_accuracy: 0.6340
Epoch 4/5
782/782 [==============================] - 559s 715ms/step - loss: 1.1099 - accuracy: 0.6112 - val_loss: 0.9680 - val_accuracy: 0.6600
Epoch 5/5
782/782 [==============================] - 602s 769ms/step - loss: 1.0640 - accuracy: 0.6290 - val_loss: 0.9324 - val_accuracy: 0.6865
сохранить обученную модель как C:\Users\fini1\Neural_network\lesson_4\saved_models\keras_cifar10_trained_model.h5 
10000/10000 [===========

## Выводы

1.

Ухудшает работу нейронной сети: не использование data_augmentation, некорректный выбор параметров data_augmentation, уменьшение 
параметра learning_rate и увеличение batch_size свыше 64. Применение datagen.fit(x_train) сильно ухудшает работу модели.

Улучшает работу нейронной сети: использование data_augmentation, увеличение epochs, увеличение num_predictions=50, batch_size=64, и запуск data augmentation через fit_generator.


2.

MNIST является более простым датасетом для распознования, поэтому изменений можно не вносить. А датасеты CIFAR100 и IMAGENET более сложные и всвязи с этим потребовалось бы внести изменения в установки параметров сети, в архитектуру слоёв нейронной сети, а также внести изменения в количество распозноваемых классов, а также скорость обучения. Возможно потребовалось бы внесение изменений в data augmentation.